In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob # Novo módulo para listar arquivos
import os

# --- 1. CONFIGURAÇÃO DE CAMINHOS E MAPEAMENTO ---
RAW_DIR = '../data/raw'
COLUNAS_PARA_RENOMEAR = {
    'OCORRENCIA_ANO': 'ano',                     
    'NATUREZA1_DESCRICAO': 'tipo_crime',       
    'ATENDIMENTO_BAIRRO_NOME': 'bairro',        
    'OCORRENCIA_DATA': 'data_completa'          
}

# --- 2. LEITURA E CONCATENAÇÃO DE TODOS OS ARQUIVOS RAW ---
print("Lendo e unificando todos os arquivos da pasta RAW...")

# Lista todos os arquivos .csv na pasta raw
all_files = glob.glob(os.path.join(RAW_DIR, "*.csv"))

list_df = []

for filename in all_files:
    try:
        # Lendo cada arquivo com a codificação e separador corretos
        df_temp = pd.read_csv(filename, sep=";", encoding="latin-1", low_memory=False)
        list_df.append(df_temp)
    except Exception as e:
        print(f"Erro ao ler {filename}: {e}")
        continue

# Concatena todos os DataFrames em um só
if list_df:
    df = pd.concat(list_df, ignore_index=True)
    print(f"✅ Total de dados unificados: {len(df)} linhas")
else:
    print("❌ Não foi possível carregar nenhum arquivo CSV.")
    # Crie um DataFrame vazio para evitar erros
    df = pd.DataFrame() 

# --- 3. TRATAMENTO FINAL E PREPARAÇÃO PARA ANÁLISE ---

if not df.empty:
    # Renomeia as colunas
    df.rename(columns=COLUNAS_PARA_RENOMEAR, inplace=True)

    # Processar Data e Mês
    if 'data_completa' in df.columns:
        # Usa o formato explícito para aceleração e precisão
        df["data"] = pd.to_datetime(df["data_completa"], errors="coerce", format='%d/%m/%Y %H:%M:%S')
        df["ano"] = df["data"].dt.year # Garante que 'ano' seja o ano da data e não apenas a coluna 'OCORRENCIA_ANO'
        df["mes"] = df["data"].dt.month
        
    print(df.head())

    # --- INÍCIO DA ANÁLISE REAL ---
    # Crimes por ano
    df.groupby('ano')['tipo_crime'].count().plot(kind='bar', title='Ocorrências por ano')
    plt.show()

    # Crimes por bairro (top 10)
    if 'bairro' in df.columns:
        df['bairro'].value_counts().head(10).plot(kind='bar', title='Top 10 bairros com mais ocorrências')
        plt.show()

KeyboardInterrupt: 

In [6]:
# ----------------------------------------------------------------------
# 6. Mapa de Calor: Tipo de Crime vs. Dia da Semana
# ----------------------------------------------------------------------
print("### 6. Mapa de Calor: Tipo de Crime por Dia da Semana ###")

if 'tipo_crime' in df.columns and 'OCORRENCIA_DIA_SEMANA' in df.columns:
    # Selecionar os 5 tipos de crime mais comuns para manter a visualização limpa
    top_5_crimes = df['tipo_crime'].value_counts().head(5).index.tolist()
    df_top_crimes = df[df['tipo_crime'].isin(top_5_crimes)]
    
    # Criar a tabela dinâmica (pivot table) para o heatmap
    crime_weekday = df_top_crimes.pivot_table(
        index='tipo_crime', 
        columns='OCORRENCIA_DIA_SEMANA', 
        aggfunc='size', 
        fill_value=0
    )
    
    # Ordem correta dos dias da semana (em Português)
    dias_ordem = ['DOMINGO', 'SEGUNDA-FEIRA', 'TERÇA-FEIRA', 'QUARTA-FEIRA', 'QUINTA-FEIRA', 'SEXTA-FEIRA', 'SÁBADO']
    
    # Reordenar as colunas
    crime_weekday = crime_weekday.reindex(columns=dias_ordem, fill_value=0)
    
    plt.figure(figsize=(14, 8))
    sns.heatmap(
        crime_weekday, 
        cmap="Reds", 
        annot=True, 
        fmt="d", 
        linewidths=.5, 
        cbar_kws={'label': 'Total de Ocorrências'}
    )
    plt.title('Concentração dos Top 5 Crimes por Dia da Semana')
    plt.ylabel('Tipo de Crime')
    plt.xlabel('Dia da Semana')
    plt.yticks(rotation=0)
    plt.show()
else:
    print("⚠️ Colunas 'tipo_crime' ou 'OCORRENCIA_DIA_SEMANA' não encontradas.")

### 6. Mapa de Calor: Tipo de Crime por Dia da Semana ###
⚠️ Colunas 'tipo_crime' ou 'OCORRENCIA_DIA_SEMANA' não encontradas.


In [7]:
# ----------------------------------------------------------------------
# 9. Tabela Gráfica: Top 10 Bairros
# ----------------------------------------------------------------------
print("### 9. Top 10 Bairros com Ocorrências (Tabela Analítica) ###")

if 'bairro' in df.columns:
    # 1. Calcular a contagem e o percentual
    total_ocorrencias = len(df)
    
    # Contagem de ocorrências por bairro
    contagem_bairro = df['bairro'].value_counts()
    
    # Criar DataFrame para a tabela
    df_top_10 = pd.DataFrame(contagem_bairro.head(10))
    df_top_10.columns = ['Total de Ocorrências']
    
    # Calcular o percentual
    df_top_10['% do Total'] = (df_top_10['Total de Ocorrências'] / total_ocorrencias) * 100
    
    # 2. Formatar e exibir a tabela
    # Aplicar formatação de número inteiro e percentual com duas casas decimais
    df_top_10_formatado = df_top_10.style.format({
        'Total de Ocorrências': '{:,.0f}',
        '% do Total': '{:.2f} %'
    }).set_caption("Top 10 Bairros Mais Afetados")
    
    # Exibir o DataFrame formatado (no Jupyter, ele renderiza como tabela HTML)
    display(df_top_10_formatado)

    # Nota: Lembre-se de importar 'display' se não tiver feito antes. Se der erro, 
    # adicione 'from IPython.display import display' no topo do notebook.
else:
    print("⚠️ Coluna 'bairro' não encontrada. Impossível gerar a tabela analítica.")

### 9. Top 10 Bairros com Ocorrências (Tabela Analítica) ###
⚠️ Coluna 'bairro' não encontrada. Impossível gerar a tabela analítica.
